In [1]:
import tensorflow as tf
import numpy as np

import model.cnn as cnn
import model.rnn as rnn
import model.classifier as classifier

import data.ops
import time
import os
import shutil

# Model definition

In [2]:
tf.reset_default_graph()
batch_size = tf.placeholder_with_default(64, [], name='batch_size')
    
cnn_params = {
    'out_dims' : [10, 10, 10, 10, 10],
    'kernel_sizes' : 360,
    'pool_sizes' : 2
}
rnn_params = {
    'rnn_sizes' : [30],
    'time_steps' : 100
}
fc_params = {
    'fc_sizes' : [100, 10]
}

input_op, seq_len, label = data.ops.get_batch_producer(
    batch_size=batch_size, path='./data/train.TFRecord')

c = cnn.get_output(seq_len=seq_len, input_op=input_op, **cnn_params)
r = rnn.get_model(batch_size=batch_size, seq_len=seq_len, input_op=c, **rnn_params)
logits, pred = classifier.get_logits_and_pred(input_op=r.last_output, **fc_params)

model_name = 'largeCNN-handloss'
MODEL_PATH = '/tmp/model/' + model_name # + c.name + r.name + classifier.name
MODEL_EXISTS = os.path.exists(MODEL_PATH)
if MODEL_EXISTS:
    print('Model directory is not empty, removing old files')
    shutil.rmtree(MODEL_PATH)


CNN--cnn10x360-10x360-10x360-10x360-10x360
Tensor("CNN/Conv1/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)
Tensor("CNN/Conv2/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)
Tensor("CNN/Conv3/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)
Tensor("CNN/Conv4/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)
Tensor("CNN/Conv5/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)

RNN--rnn--steps100--sizes30
LSTMStateTuple(c=<tf.Tensor 'RNN/LSTM/dynamic_wrapper/rnn/while/Exit_2:0' shape=(?, 30) dtype=float32>, h=<tf.Tensor 'RNN/LSTM/dynamic_wrapper/rnn/while/Exit_3:0' shape=(?, 30) dtype=float32>)

FC--fc100-10
Tensor("classifier/hidden_layer0/fully_connected/Relu:0", shape=(?, 100), dtype=float32)
Tensor("classifier/hidden_layer1/fully_connected/Relu:0", shape=(?, 10), dtype=float32)
Tensor("classifier/logits/BiasAdd:0", shape=(?, 4), dtype=float32)
Tensor("classifier/predictions:0", shape=(?, 4), dtype=float32)


# Time measure

convenience function

In [3]:
def measure_time(op, feed_dict={}, n_times=10):
    with tf.Session() as sess:
        print('Sess started')
        coord = tf.train.Coordinator()
        tf.global_variables_initializer().run()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        print('Evaluating')
        for _ in range(n_times):
            t = time.time()
            fetch = sess.run(op, feed_dict)
            print(fetch, 'Eval time:', time.time() - t)
            
        print('Closing threads')
        coord.request_stop()
        coord.join(threads)

        return fetch

# Evaluation

## **Confusion matrix**

## **Accuracy operator**

In [4]:
with tf.name_scope('evaluation'):
    with tf.name_scope('one_hot_encoding'):
        y_oh = tf.cast(tf.equal(
            logits, tf.reduce_max(logits, axis=1)[:, None]), tf.float32)[..., None]

        label_oh = tf.one_hot(label, depth=4)[..., None]
    with tf.name_scope('confusion_matrix'):
        conf_op = tf.reduce_sum(tf.transpose(y_oh, perm=[0, 2, 1]) * label_oh,
            axis=0, name='result')

    with tf.name_scope('accuracy'):
        y_tot = tf.reduce_sum(conf_op, axis=0, name='label_class_sum')
        label_tot = tf.reduce_sum(conf_op, axis=1, name='pred_class_sum')
        correct_op = tf.diag_part(conf_op, name='correct_class_sum')
        eps = tf.constant([1e-10] * 4, name='eps')
        acc = tf.reduce_mean(2*correct_op / (y_tot + label_tot + eps), name='result')

# Sparse, weighted softmax loss

In [5]:
class_hist = np.load('./data/class_hist.npy')
with tf.name_scope('loss'):
    weight = tf.constant([.1, 1, .2, 3])
    #weight = tf.constant(1 - np.sqrt(class_hist/class_hist.sum()), name='weights')
    weight = tf.gather(weight, label, name='weight_selector')
    loss = tf.losses.sparse_softmax_cross_entropy(label, logits, weight, scope='weighted_loss')
    unweighted_loss = tf.losses.sparse_softmax_cross_entropy(label, logits, scope='unweighted_loss')
class_hist, weight

(array([5154,  771, 2557,   46]),
 <tf.Tensor 'loss/weight_selector:0' shape=(?,) dtype=float32>)

# Train operator

In [6]:
with tf.name_scope('train'):
    learning_rate = tf.Variable(initial_value=.05, trainable=False, name='learning_rate')
    global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
    grad_clip = tf.Variable(initial_value=3., trainable=False, name='grad_clip')
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    gvs = optimizer.compute_gradients(loss)
    with tf.name_scope('gradient_clipping'):
        capped_gvs = [(tf.clip_by_value(grad, -grad_clip, grad_clip), var) 
                      for grad, var in gvs]
        
    opt = optimizer.apply_gradients(capped_gvs, global_step)

# Summaries

In [7]:
train_writer = tf.summary.FileWriter(MODEL_PATH, graph=tf.get_default_graph())
sum_ops = []
for v in tf.trainable_variables():
    sum_ops.append(tf.summary.histogram(v.name[:-2], v))
    sum_ops.append(tf.summary.histogram('gradients/'+v.name[:-2], tf.gradients(loss, v)))

sum_ops.append(tf.summary.scalar('weighted_loss', loss))
sum_ops.append(tf.summary.scalar('unweighted_loss', unweighted_loss))
sum_ops.append(tf.summary.scalar('accuracy', acc))
sum_ops.append(tf.summary.image('confusion_matrix', conf_op[None, ..., None], max_outputs=10))
summaries = tf.summary.merge(sum_ops)



In [ ]:
saver = tf.train.Saver(keep_checkpoint_every_n_hours=1)
#with open('test.txt', 'w') as f:
    #metagraph = saver.export_meta_graph(as_text=True)
    #f.write(str(metagraph.ListFields()))

In [ ]:
TRAIN_STEPS = 5000
with tf.Session() as sess:
    print('Sess started')
    
    print('Initializing model')
    tf.global_variables_initializer().run()
        
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    print('Training')
    for i in range(TRAIN_STEPS):
        t = time.time()
        fetch = sess.run([opt, loss, acc, global_step])
        step = fetch[-1]
        print('%d/%d'%(step, TRAIN_STEPS), 
              'time:%f'%(time.time()-t), 
              'loss:%f'%fetch[1],
              'acc:%f'%fetch[2]
              )
        if step % 10 == 0:
            print('Evaluating summaries...')
            train_writer.add_summary(summaries.eval(), global_step=fetch[-1])
        if step % 50 == 0:
            print('Saving model...')
            print(saver.save(sess, MODEL_PATH, global_step=fetch[-1]))
    
    print('Ending, closing producer threads')
    coord.request_stop()
    coord.join(threads)

Sess started
Initializing model
Training
1/4999 time:4.857495 loss:0.286065 acc:0.148516
2/4999 time:3.684704 loss:0.370896 acc:0.188148
3/4999 time:1.284986 loss:0.285891 acc:0.238073
4/4999 time:1.331241 loss:0.356416 acc:0.224567
5/4999 time:1.270860 loss:0.244619 acc:0.138799
6/4999 time:1.313504 loss:0.221535 acc:0.039447
7/4999 time:1.312521 loss:0.297948 acc:0.079837
8/4999 time:1.309821 loss:0.264745 acc:0.054894
9/4999 time:1.275188 loss:0.265898 acc:0.058290
10/4999 time:1.303985 loss:0.573987 acc:0.183614
Evaluating summaries...
11/4999 time:1.381268 loss:0.239877 acc:0.096300
12/4999 time:1.404033 loss:0.228286 acc:0.048856
13/4999 time:1.339311 loss:0.275151 acc:0.059989
14/4999 time:1.281811 loss:0.263105 acc:0.109855
15/4999 time:1.325436 loss:0.445003 acc:0.105072
16/4999 time:1.393899 loss:0.397016 acc:0.086350
17/4999 time:1.358587 loss:0.236394 acc:0.034441
18/4999 time:1.319567 loss:0.251169 acc:0.056636
19/4999 time:3.633179 loss:0.361393 acc:0.049296
20/4999 time: